In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraModel, LoraConfig

/home/zcwang/.conda/envs/trace/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zcwang/.conda/envs/trace/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
model_name_or_path = "/home/zcwang/data/model/google/gemma-2b-it"

config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],
)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

lora_model = LoraModel(model, config, adapter_name="task_0")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.94it/s]



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/zcwang/.conda/envs/trace/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/zcwang/.conda/envs/trace/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


In [3]:
lora_model

LoraModel(
  (model): GemmaForCausalLM(
    (model): GemmaModel(
      (embed_tokens): Embedding(256000, 2048, padding_idx=0)
      (layers): ModuleList(
        (0-17): 18 x GemmaDecoderLayer(
          (self_attn): GemmaSdpaAttention(
            (q_proj): lora.Linear(
              (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
              (lora_dropout): ModuleDict(
                (task_0): Identity()
              )
              (lora_A): ModuleDict(
                (task_0): Linear(in_features=2048, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (task_0): Linear(in_features=8, out_features=2048, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (k_proj): Linear(in_features=2048, out_features=256, bias=False)
            (v_proj): lora.Linear(
              (base_layer): Linear(in_features=2048, out_feat

In [4]:
lora_model.base_model

GemmaModel(
  (embed_tokens): Embedding(256000, 2048, padding_idx=0)
  (layers): ModuleList(
    (0-17): 18 x GemmaDecoderLayer(
      (self_attn): GemmaSdpaAttention(
        (q_proj): lora.Linear(
          (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
          (lora_dropout): ModuleDict(
            (task_0): Identity()
          )
          (lora_A): ModuleDict(
            (task_0): Linear(in_features=2048, out_features=8, bias=False)
          )
          (lora_B): ModuleDict(
            (task_0): Linear(in_features=8, out_features=2048, bias=False)
          )
          (lora_embedding_A): ParameterDict()
          (lora_embedding_B): ParameterDict()
        )
        (k_proj): Linear(in_features=2048, out_features=256, bias=False)
        (v_proj): lora.Linear(
          (base_layer): Linear(in_features=2048, out_features=256, bias=False)
          (lora_dropout): ModuleDict(
            (task_0): Identity()
          )
          (lora_A): ModuleDict(


In [5]:
lora_model.save_pretrained("/home/zcwang/data/model/google/gemma-2b-it-lora") # save the whole model

In [7]:
lora_model.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('model',
               GemmaForCausalLM(
                 (model): GemmaModel(
                   (embed_tokens): Embedding(256000, 2048, padding_idx=0)
                   (layers): ModuleList(
                     (0-17): 18 x GemmaDecoderLayer(
                       (self_attn): GemmaSdpaAttention(
                         (q_proj): lora.Linear(
          

In [6]:
from peft import PeftModel
assert isinstance(lora_model, PeftModel)

AssertionError: 

In [10]:
lora_model.add_weighted_adapter("task_0", 0.5, "task_1", combination_type="cat")

ValueError: Adapter t does not exist

In [ ]:
lora_model.peft_config.keys()

dict_keys(['task_0'])

In [18]:
from dataclasses import replace

lora_model.model.peft_config["task_1"] = replace(lora_model.model.peft_config["task_0"], lora_alpha=10)
lora_model.inject_adapter(lora_model.model, "task_1")

In [22]:
lora_model.model.peft_config["task_2"] = replace(lora_model.model.peft_config["task_0"], r=10)
lora_model.inject_adapter(lora_model.model, "task_2")

In [23]:
lora_model

LoraModel(
  (model): GemmaForCausalLM(
    (model): GemmaModel(
      (embed_tokens): Embedding(256000, 2048, padding_idx=0)
      (layers): ModuleList(
        (0-17): 18 x GemmaDecoderLayer(
          (self_attn): GemmaSdpaAttention(
            (q_proj): lora.Linear(
              (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
              (lora_dropout): ModuleDict(
                (task_0): Identity()
                (task_1): Identity()
                (task_2): Identity()
              )
              (lora_A): ModuleDict(
                (task_0): Linear(in_features=2048, out_features=8, bias=False)
                (task_1): Linear(in_features=2048, out_features=8, bias=False)
                (task_2): Linear(in_features=2048, out_features=10, bias=False)
              )
              (lora_B): ModuleDict(
                (task_0): Linear(in_features=8, out_features=2048, bias=False)
                (task_1): Linear(in_features=8, out_features=2048, bia

In [26]:
lora_model.active_adapter

'task_0'

In [27]:
lora_model.active_adapters

['task_0']

In [ ]:
lora_model.active_adapters

In [45]:
lora_model.set_adapter([])

In [47]:
lora_model.active_adapter

[]

In [46]:
lora_model.active_adapter

[]

In [34]:
lora_model

LoraModel(
  (model): GemmaForCausalLM(
    (model): GemmaModel(
      (embed_tokens): Embedding(256000, 2048, padding_idx=0)
      (layers): ModuleList(
        (0-17): 18 x GemmaDecoderLayer(
          (self_attn): GemmaSdpaAttention(
            (q_proj): lora.Linear(
              (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
              (lora_dropout): ModuleDict(
                (task_0): Identity()
                (task_1): Identity()
                (task_2): Identity()
              )
              (lora_A): ModuleDict(
                (task_0): Linear(in_features=2048, out_features=8, bias=False)
                (task_1): Linear(in_features=2048, out_features=8, bias=False)
                (task_2): Linear(in_features=2048, out_features=10, bias=False)
              )
              (lora_B): ModuleDict(
                (task_0): Linear(in_features=8, out_features=2048, bias=False)
                (task_1): Linear(in_features=8, out_features=2048, bia

In [35]:
lora_model.model.peft_config

{'task_0': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False),
 'task_1': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj'}, lora_alpha=10, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False

In [38]:
replace(lora_model.model.peft_config["task_0"], r=20)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=20, target_modules={'v_proj', 'q_proj'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [39]:
lora_model.model.peft_config

{'task_0': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False),
 'task_1': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj'}, lora_alpha=10, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False

In [43]:
lora_model.config

GemmaConfig {
  "_name_or_path": "/home/zcwang/data/model/google/gemma-2b-it",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.0",
  "use_cache": true,
  "vocab_size": 256000
}